# Building Optimizer Model

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("cleaned_data/uber_demand_travel_times.csv")
df.head()

,nhood,day_of_week,hour,avg_pickups,max_pickups,min_pickups,avg_travel_time,min_travel_time,max_travel_time,nhood_id
0,Castro/Upper Market,0,3,0.80,3.2,0.0,703.89,35.6,1981.40,0
1,Castro/Upper Market,0,4,0.95,3.0,0.0,703.89,35.6,1981.40,0
2,Castro/Upper Market,0,5,1.97,7.4,0.4,703.89,35.6,1981.40,0
3,Castro/Upper Market,0,6,3.02,7.6,0.0,703.89,35.6,1981.40,0
4,Castro/Upper Market,0,7,6.20,13.6,1.0,743.23,48.8,2032.79,0


In [3]:
example_one = {
    # Available to work at max 12 hours per week
    "max_total_hours": 12,
    # Monday, Tuesday, Thursday, Friday
    "day_available": [0, 1, 3, 4],
    # 4m-8pm
    "time_available": [16, 17, 18, 19],
    # Castro/Upper Market, Mission, Western SoMa
    "location_available": [0, 9, 18]
}

In [4]:
df_filter = df[df["hour"].isin(example_one["time_available"]) & 
                          df["day_of_week"].isin(example_one["day_available"]) &
                          df["nhood_id"].isin(example_one["location_available"])].reset_index()

df_filter.head()

,index,nhood,day_of_week,hour,avg_pickups,max_pickups,min_pickups,avg_travel_time,min_travel_time,max_travel_time,nhood_id
0,13,Castro/Upper Market,0,16,7.07,15.0,1.8,738.62,32.00,1911.67,0
1,14,Castro/Upper Market,0,17,10.30,22.2,4.0,738.62,32.00,1911.67,0
2,15,Castro/Upper Market,0,18,14.34,38.8,4.6,738.62,32.00,1911.67,0
3,16,Castro/Upper Market,0,19,13.60,34.4,3.0,738.62,32.00,1911.67,0
4,37,Castro/Upper Market,1,16,8.53,21.0,3.8,793.75,25.57,1992.00,0


In [5]:
def get_probablity_new_trip(d_max, d_min, d_avg):
    """
    :param d_max maximum pickup counts for current region and time block.
    :param d_min minimum pickup counts for current region and time block.
    :param d_avg average pickup counts for current region and time block.
    :return: probability new trip happens, range between 0.5 and 1
    """
    p_min = 0.5
    p_max = 1
    return p_min + ((p_max-p_min)/(d_max - d_min)) * (d_avg - d_min)

In [6]:
average_hourly_wage = 22

# TODO: Prob can switch to matrix multiplication for faster?
def get_expected_revenue(day_available, time_available, location_available):
    """
    Return expected revenue from probability of new trip and minimum of trips can be taken within 1 hour
    :param day_available:
    :return:
    For example: {
        0: [28, ....],
        3: [21, ....],
        4: [18, ....],
    }
    """
    expected_revenue = np.zeros(shape=(7,24,19))
    for day in day_available:
        for hour in time_available:
            for location in location_available:
                df_filter_row = df_filter[(df_filter["day_of_week"] == day) &
                                          (df_filter["hour"] == hour) &
                                         (df_filter["nhood_id"] == location)]
            
                # This is min number of trips per hour
                min_num_trips = 3600/df_filter_row["max_travel_time"].values[0]
                probability_new_trip = get_probablity_new_trip(df_filter_row["max_pickups"].values[0],
                                                               df_filter_row["min_pickups"].values[0],
                                                               df_filter_row["avg_pickups"].values[0])
                expected_revenue[day][hour][location] = probability_new_trip*min_num_trips*average_hourly_wage
    return expected_revenue

In [7]:
# Calculate the expected revenue
expected_revenue = get_expected_revenue(example_one["day_available"], example_one["time_available"],
                                       example_one["location_available"])
expected_revenue

array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       ...,

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0.

In [8]:
from pulp import LpMaximize, LpProblem, LpVariable, lpSum, LpBinary, LpConstraint

# Initialize
# There's 19 neighborhoods for j and 24 time blocks for i.
# There's 7 days of a week
dim_j = range(0, 19)
dim_i = range(0, 24)
dim_k = range(0, 7)

# time_available_{i}=1 indicates that the driver is available to drive at time block i, and 0 otherwise.
time_available = [0] * 24
for time_slot in example_one["time_available"]:
    time_available[time_slot] = 1

# location_available_{j} = 1 indicate the driver is willing to drive in nhood j, and 0 otherwise.
location_available = [0] * 19
for nhood in example_one["location_available"]:
    location_available[nhood] = 1
    
# day_available_{k} = 1 indicates the driver is available on day k, and 0 otherwise
day_available = [0] * 7
for day in example_one["day_available"]:
    day_available[day] = 1

# Create the model
model = LpProblem(name="uber-scheduling", sense=LpMaximize)

# The decision variables x is a matrix of size 7x19x24
x = LpVariable.matrix('x', (dim_k, dim_i, dim_j) , 0, 1, LpBinary)

# Add the constraints
# Constraint 1: Time Availability of driver
# Constrain 2: Location Availablity of the driver
# Constrain 3: Day Available of the driver
for k in dim_k:
    for i in dim_i:
        for j in dim_j:
            model += (x[k][i][j] <= time_available[i], f"Time Availability Constraint: {k} {i} {j}")
            model += (x[k][i][j] <= location_available[j], f"Location Availability Constraint: {k} {i} {j}")
            model += (x[k][i][j] <= day_available[k], f"Day Availability Constraint: {k} {i} {j}")

# Constraint 4: Maximum Time Per Week Constraint
model += (lpSum([x[k][i][j] for k in dim_k for i in dim_i for j in dim_j]) <= example_one["max_total_hours"], "Maximum Time Per Week Constraint")

# Constraint 5: Only work at one neighborhood at a time
for k in dim_k:
    for i in dim_i:
        model += (lpSum([x[k][i][j] for j in dim_j]) <= 1, f"Only work at one neighborhood at a time: {k} {i}")

# Objective function: Maximize revenue
model += lpSum([expected_revenue[k]*x[k] for k in dim_k])

# Solve
status = model.solve()

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /usr/local/Cellar/jupyterlab/3.3.2/libexec/lib/python3.9/site-packages/pulp/apis/../solverdir/cbc/osx/64/cbc /var/folders/xf/p6q8333s13xgb0xlq9dw5l3w0000gp/T/8a699d50091c4382acc6a54b5d7533c3-pulp.mps max timeMode elapsed branch printingOptions all solution /var/folders/xf/p6q8333s13xgb0xlq9dw5l3w0000gp/T/8a699d50091c4382acc6a54b5d7533c3-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 9750 COLUMNS
At line 32143 RHS
At line 41889 BOUNDS
At line 45082 ENDATA
Problem MODEL has 9745 rows, 3192 columns and 15960 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 333.525 - 0.00 seconds
Cgl0002I 3144 variables fixed
Cgl0004I processed model has 1 rows, 16 columns (16 integer (16 of which binary)) and 16 elements
Cbc0038I Initial state - 0 integers unsatisfied sum - 0
Cbc0038I Solution found of -333.52

In [9]:
print(f"Objective Value: {model.objective.value()}")

Objective Value: 333.52499798918313


In [10]:
day_of_week_map = {
    0: 'Monday',
    1: 'Tuesday',
    2: 'Wednesday',
    3: 'Thursday',
    4: 'Friday',
    5: 'Saturday',
    6: 'Sunday'
}
neighborhood_map = {0: 'Castro/Upper Market',
                    1: 'Chinatown',
                    2: 'Civic Center',
                    3: 'Eastern SoMa',
                    4: 'Financial District',
                    5: 'Haight Ashbury/Cole Valley',
                    6: 'Hayes Valley/Lower Haight',
                    7: 'Inner Richmond',
                    8: 'Marina/Cow Hollow',
                    9: 'Mission',
                    10: 'Nob Hill',
                    11: 'North Beach/Waterfront',
                    12: 'Pacific Heights',
                    13: 'Potrero Hill',
                    14: 'Russian Hill',
                    15: 'Tenderloin',
                    16: 'Union Square',
                    17: 'Western Addition',
                    18: 'Western SoMa'}

print('Intepret schedule as: \n')
for var in model.variables():
    if var.value() == 1:
        var_nums = [int(num) for num in var.name.split('_')[1:]]
        print(f"On {day_of_week_map[var_nums[0]]}, at time {var_nums[1]}, works at neighborhood {neighborhood_map[var_nums[2]]} ")

Intepret schedule as: 

On Monday, at time 16, works at neighborhood Castro/Upper Market 
On Monday, at time 17, works at neighborhood Castro/Upper Market 
On Monday, at time 19, works at neighborhood Castro/Upper Market 
On Tuesday, at time 16, works at neighborhood Western SoMa 
On Tuesday, at time 17, works at neighborhood Castro/Upper Market 
On Tuesday, at time 18, works at neighborhood Castro/Upper Market 
On Tuesday, at time 19, works at neighborhood Castro/Upper Market 
On Thursday, at time 19, works at neighborhood Western SoMa 
On Friday, at time 16, works at neighborhood Castro/Upper Market 
On Friday, at time 17, works at neighborhood Castro/Upper Market 
On Friday, at time 18, works at neighborhood Castro/Upper Market 
On Friday, at time 19, works at neighborhood Castro/Upper Market 
